In [1]:
import pandas as pd
import Levenshtein
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer
from fuzzywuzzy import fuzz
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
import torch
# from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
from datasets import Dataset

d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_json('../data/final_data/temp.jsonl', lines=True)

# Without Context

In [3]:
bert_without_context = data[['c1','c2','match']]
bert_without_context

,c1,c2,match
0,anz,ANZ,1
1,anz,Australian New Zealand Banking Group,1
2,anz,A.N.Z.,1
3,anz,ANZ Bank,1
4,anz,Anz,1
...,...,...,...
23229,aerotek,AeroVironment,0
23230,aerotek,Aeropostale,0
23231,michigan state university,Michigan Technological University,0
23232,michigan state university,Central Michigan University,0


In [4]:
# Combine c1 and c2 into a single string with a separator
bert_without_context['combined'] = bert_without_context['c1'] + " [SEP] " + bert_without_context['c2']

# Split into training and testing sets
train_df, test_df = train_test_split(bert_without_context, test_size=0.2, random_state=42)

# Prepare for HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df[['combined', 'match']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['combined', 'match']].reset_index(drop=True))

# Load BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    # Tokenisasi teks
    tokens = tokenizer(examples['combined'], padding='max_length', truncation=True)
    # Tambahkan label ke token output
    tokens['labels'] = examples['match']
    return tokens

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


C:\Users\mfauz\AppData\Local\Temp\ipykernel_1996\1177542444.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bert_without_context['combined'] = bert_without_context['c1'] + " [SEP] " + bert_without_context['c2']
d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 4647/4647 [00:01<00:00, 3424.46 examples/s]


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model with classification head
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir='../data/results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,  # Optional, jika ingin menggunakan tokenizer secara eksplisit
)

# Fine-tune the model
trainer.train()

In [ ]:
trainer.evaluate()

## Evaluate Model

In [5]:
# Load model dan tokenizer dari direktori yang sudah disimpan
model = DistilBertForSequenceClassification.from_pretrained("../model/distillbert_no_context")
tokenizer = DistilBertTokenizer.from_pretrained("../model/distillbert_no_context")

In [6]:
import evaluate

# Load metric
accuracy_metric = evaluate.load("accuracy")

# Set arguments untuk evaluasi saja
training_args = TrainingArguments(
    output_dir='./results',                # Directory untuk menyimpan hasil evaluasi (misalnya metrics)
    per_device_eval_batch_size=8,          # Ukuran batch untuk evaluasi
    do_train=False,                        # Jangan lakukan training
    do_eval=True,                          # Hanya lakukan evaluasi
    evaluation_strategy="no",              # Evaluasi tidak per epoch karena Anda tidak sedang melatih ulang
    logging_dir='./logs',                  # Directory untuk menyimpan logs
    report_to="none"                       # Tidak perlu logging ke tempat lain (misalnya WandB)
)

# Define the function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy


trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate()

d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 581/581 [01:19<00:00,  7.31it/s]


In [7]:
print(results)

{'eval_loss': 0.37126779556274414, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.9162900796212611, 'eval_runtime': 80.3665, 'eval_samples_per_second': 57.823, 'eval_steps_per_second': 7.229}


## Inference Test

In [ ]:
# Siapkan input teks untuk prediksi
c1 = "bank niaga"
c2 = "cimb niaga"

# Gabungkan c1 dan c2 dengan separator seperti yang digunakan selama pelatihan
input_text = c1 + " [SEP] " + c2

# Tokenisasi teks
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

# Lakukan inference (pastikan model dalam mode evaluasi)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Ambil logits dari output
logits = outputs.logits

# Konversi logits ke prediksi kelas (0 atau 1)
predictions = torch.argmax(logits, dim=-1)

# Lihat hasil prediksi
print(f"Prediksi: {predictions.item()}")  # Prediksi bisa 0 atau 1

## Save Model

In [ ]:
model.save_pretrained("../model/distillbert_no_context")
tokenizer.save_pretrained("../model/distillbert_no_context")

# With Context

In [8]:
bert_with_context = data[['c1','c2','c1_context','c2_context','match']]
bert_with_context

,c1,c2,c1_context,c2_context,match
0,anz,ANZ,The Australia and New Zealand Banking Group Li...,We provide banking and financial products and ...,1
1,anz,Australian New Zealand Banking Group,The Australia and New Zealand Banking Group Li...,ANZ\r\n- The Australian New Zealand Banking Gr...,1
2,anz,A.N.Z.,The Australia and New Zealand Banking Group Li...,We provide banking and financial products and ...,1
3,anz,ANZ Bank,The Australia and New Zealand Banking Group Li...,ANZ Bank New Zealand Limited operates as a ban...,1
4,anz,Anz,The Australia and New Zealand Banking Group Li...,We provide banking and financial products and ...,1
...,...,...,...,...,...
23229,aerotek,AeroVironment,Aerotek\nprovides staffing and services soluti...,AeroVironment (NASDAQ: AVAV) is a technology s...,0
23230,aerotek,Aeropostale,Aerotek\nprovides staffing and services soluti...,Aéropostale is a specialty retailer of high-qu...,0
23231,michigan state university,Michigan Technological University,Michigan State University is the nation's prem...,Michigan Technological University is a flagshi...,0
23232,michigan state university,Central Michigan University,Michigan State University is the nation's prem...,Central Michigan University is a leading publi...,0


In [9]:
# Combine c1: context and c2: context into a single string with a separator
bert_with_context['combined'] = bert_with_context['c1'] + ": " + bert_with_context['c1_context']\
+ " [SEP] " + bert_with_context['c2'] + ": " + bert_with_context['c2_context']

# Split into training and testing sets
train_df, test_df = train_test_split(bert_with_context, test_size=0.2, random_state=42)

# Prepare for HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df[['combined', 'match']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['combined', 'match']].reset_index(drop=True))

# Load BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    # Tokenisasi teks
    tokens = tokenizer(examples['combined'], padding='max_length', truncation=True)
    # Tambahkan label ke token output
    tokens['labels'] = examples['match']
    return tokens

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


C:\Users\mfauz\AppData\Local\Temp\ipykernel_3412\1327821698.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bert_with_context['combined'] = bert_with_context['c1'] + ": " + bert_with_context['c1_context']\
d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 4647/4647 [00:08<00:00, 569.97 examples/s]


In [10]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model with classification head
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir='../data/results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer, 
)

# Fine-tune the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  7%|▋         | 500/6972 [53:55<11:23:19,  6.33s/it]

{'loss': 0.4462, 'grad_norm': 11.03742790222168, 'learning_rate': 1.8565691336775675e-05, 'epoch': 0.22}


 14%|█▍        | 1000/6972 [2:21:50<17:57:39, 10.83s/it]

{'loss': 0.2636, 'grad_norm': 0.7353519797325134, 'learning_rate': 1.7131382673551347e-05, 'epoch': 0.43}


 22%|██▏       | 1500/6972 [3:51:58<16:35:57, 10.92s/it]

...

{'loss': 0.0668, 'grad_norm': 0.023077568039298058, 'learning_rate': 1.3539873780837638e-06, 'epoch': 2.8}


                                                        
100%|██████████| 6972/6972 [20:47:31<00:00,  7.74s/it]

{'eval_loss': 0.16030742228031158, 'eval_runtime': 1310.8114, 'eval_samples_per_second': 3.545, 'eval_steps_per_second': 0.443, 'epoch': 3.0}
{'train_runtime': 74851.1349, 'train_samples_per_second': 0.745, 'train_steps_per_second': 0.093, 'train_loss': 0.15340027040403872, 'epoch': 3.0}


100%|██████████| 6972/6972 [20:47:31<00:00, 10.74s/it]


TrainOutput(global_step=6972, training_loss=0.15340027040403872, metrics={'train_runtime': 74851.1349, 'train_samples_per_second': 0.745, 'train_steps_per_second': 0.093, 'total_flos': 7386514616457216.0, 'train_loss': 0.15340027040403872, 'epoch': 3.0})

## Save Model

In [12]:
model.save_pretrained("../model/distillbert_with_context")
tokenizer.save_pretrained("../model/distillbert_with_context")

('../model/distillbert_with_context\\tokenizer_config.json',
 '../model/distillbert_with_context\\special_tokens_map.json',
 '../model/distillbert_with_context\\vocab.txt',
 '../model/distillbert_with_context\\added_tokens.json')

## Evaluate Model

In [13]:
import evaluate

# Load metric
accuracy_metric = evaluate.load("accuracy")

# Set arguments untuk evaluasi saja
training_args = TrainingArguments(
    output_dir='./results',                # Directory untuk menyimpan hasil evaluasi (misalnya metrics)
    per_device_eval_batch_size=8,          # Ukuran batch untuk evaluasi
    do_train=False,                        # Jangan lakukan training
    do_eval=True,                          # Hanya lakukan evaluasi
    evaluation_strategy="no",              # Evaluasi tidak per epoch karena Anda tidak sedang melatih ulang
    logging_dir='./logs',                  # Directory untuk menyimpan logs
    report_to="none"                       # Tidak perlu logging ke tempat lain (misalnya WandB)
)

# Define the function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy


trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate()

d:\Fauzan\anaconda3\envs\company-matching\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 581/581 [21:46<00:00,  2.25s/it]


In [14]:
print(results)

{'eval_loss': 0.16030742228031158, 'eval_model_preparation_time': 0.0029, 'eval_accuracy': 0.9685818807833011, 'eval_runtime': 1308.7915, 'eval_samples_per_second': 3.551, 'eval_steps_per_second': 0.444}
